In [1]:
import pandas as pd
import numpy as np


In [2]:
#import data
df = pd.read_csv('df_ada_embedded_floats_CC1204.csv')

In [3]:
y = df['BloodGlucose']
data2 = df.iloc[:,:17]
df_embedded = df.iloc[:,1:]
embeddings = df.iloc[:,18:]

In [4]:
df.iloc[:,17:].head(2)

,DR1TP182,BloodGlucose,0,1,2,3,4,5,6,7,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,93.518676,78.110787,-0.001138,-0.005256,0.038269,-0.022218,0.003684,0.005965,-0.011949,-0.007395,...,-0.006982,0.007166,0.039187,-0.029108,0.012481,-0.033072,-0.014239,-0.031234,-0.026562,-0.003858
1,80.883578,55.974437,-0.002109,-0.005945,0.039484,-0.020860,0.004337,0.004245,-0.014271,-0.009042,...,-0.004373,0.006938,0.039247,-0.027489,0.012423,-0.034381,-0.012883,-0.030225,-0.025621,-0.004429


The following chunk of code initializes a DML model that includes both the OUTCOME and the TREATMENT regression.

In [5]:
%pip install econml scikit-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
import xgboost
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from econml.dml import DML
from sklearn.model_selection import KFold

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


# Standardize Data

In [7]:
y = df['BloodGlucose']
T = df['Treatment']
df2 = df.copy()
df2.drop(columns=['Treatment', 'BloodGlucose'], inplace = True)
data2 = df.iloc[:, 1:16]
embeddings = df.iloc[:, 16:]

from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Standardize tabular data (data2)
scaler = StandardScaler()
tab_data = scaler.fit_transform(data2)

# L2 normalization for embeddings
embedding_magnitudes = np.linalg.norm(embeddings, axis=1, keepdims=True) + 1e-8
normalized_embeddings = embeddings / embedding_magnitudes



# non
df_none = pd.concat([pd.DataFrame(data2, columns=data2.columns),
                           pd.DataFrame(embeddings, columns=embeddings.columns)], axis=1)


# Combine standardized tabular data and normalized embeddings
df_embedded = pd.concat([pd.DataFrame(tab_data, columns=data2.columns),
                           pd.DataFrame(normalized_embeddings, columns=embeddings.columns)], axis=1)




# Standardize embeddings

standardized_embeddings = scaler.fit_transform(embeddings)
# Combine standardized tabular data and normalized embeddings
df_embedded_std = pd.concat([pd.DataFrame(tab_data, columns=data2.columns),
                           pd.DataFrame(standardized_embeddings, columns=embeddings.columns)], axis=1)


# Full

# Combine standardized tabular data and normalized embeddings
df_embedded_full = pd.concat([pd.DataFrame(tab_data, columns=data2.columns),
                           pd.DataFrame(embeddings, columns=embeddings.columns)], axis=1)


tb = pd.DataFrame(tab_data, columns=data2.columns)

In [8]:
X_embedded = df_embedded
X_non_embedded = data2

# XGBoost

# Grid Search for Outcome Model

In [9]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import torch


xgb_gridsearch = XGBRegressor(objective="reg:squarederror", random_state=42, tree_method='hist', device='cuda')


def wrapped_mse(y_true, y_pred, **kwargs):
    # Check if inputs are PyTorch tensors
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.cpu().numpy()
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.cpu().numpy()

    # Call sklearn's mean_squared_error
    return mean_squared_error(y_true, y_pred, **kwargs)


X = torch.Tensor(df_none.to_numpy()).cuda()
y = torch.Tensor(y.to_numpy()).cuda()

# Define the parameter grid
# Define the hyperparameter grid including min_child_weight
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'min_child_weight': [1, 5, 10],
}

# Define the cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric (negative MSE for minimization)
scorer = make_scorer(wrapped_mse, greater_is_better=False)

# Perform grid search
grid_search = GridSearchCV(
    estimator=xgb_gridsearch,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv,
    verbose=1,
    n_jobs=8,  # Parallelize
    # error_score="raise",
)

# Fit grid search
grid_search.fit(X, y)

# Print best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Negative MSE):", grid_search.best_score_)

Fitting 5 folds for each of 750 candidates, totalling 3750 fits



232 fits failed out of a total of 3750.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tlilly/miniconda3/envs/rb_project/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/tlilly/miniconda3/envs/rb_project/lib/python3.12/site-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "/home/tlilly/miniconda3/envs/rb_project/lib/python3.12/site-packages/xgboost/sklearn.py", line 1108, in fit
    self._Booster = train(
                    ^^^^^^
  File "/home/tlilly/miniconda3/envs/rb_project/l

Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 1.0}
Best Score (Negative MSE): -0.2253224864602089


# Grid Search for Treatment Model

In [10]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.metrics import log_loss


# Example treatment data
X = torch.Tensor(df_none.to_numpy()).cuda()  # Feature matrix
T = torch.Tensor(T.to_numpy()).cuda()         # Treatment vector (binary)

scorer_continuous = make_scorer(wrapped_mse, greater_is_better=False)

xgb_treatment_model = XGBRegressor(objective="reg:squarederror", random_state=42, tree_method='hist', device='cuda')

# Define the parameter grid
param_grid_treatment = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'min_child_weight': [1, 5, 10],
}

# Define the cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric (log loss for classification)
scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# Perform grid search
grid_search_treatment = GridSearchCV(
    estimator=xgb_treatment_model,
    param_grid=param_grid_treatment,
    scoring=scorer,
    cv=cv,
    verbose=1,
    n_jobs=8,  # Parallelize
    # error_score="raise",
)

# Fit grid search
grid_search_treatment.fit(X, T)

# Print best parameters and score
print("Best Parameters for Treatment Model:", grid_search_treatment.best_params_)
print("Best Score (Negative Log Loss):", grid_search_treatment.best_score_)

Fitting 5 folds for each of 750 candidates, totalling 3750 fits


The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
/home/tlilly/miniconda3/envs/rb_project/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/tlilly/miniconda3/envs/rb_project/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tlilly/miniconda3/envs/rb_project/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Best Parameters for Treatment Model: {'learning_rate': 0.001, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 0.8}
Best Score (Negative Log Loss): nan


One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan